In [20]:
import requests as req

In [21]:
import re

In [69]:
from db import DB

In [24]:
headers = {"Referer": "http://lbsbbs.amap.com/login.php?callback=http://lbsbbs.amap.com/forum.php?mod=viewthread&tid=28659"}

In [25]:
res = req.get(url, headers = headers)

In [26]:
rePost.findall(res.text)[0].strip()

'昨天我到火车站，打开导航。他选的线路是最近的，但我知道是最堵的线。所以我选择走环线，车开了一半，导航死活不改路线，总是提醒我前方调头。等我开到快到火车站的时候，还在要我调头，这系统有什么用，被这导航搞得太郁闷了。前提是以经更新最新版了。 你们有这种情况吗？'

In [71]:
reItem = re.compile(r'<tbody id="normalthread_(?P<tid>\d+)">(?P<item>[\s\S]*?)<\/tbody>')
reTitleTh = re.compile(r'<th class="new">([\s\S]*?)<\/th>')
reEm = re.compile(r'<em>([\s\S]*?)<\/em>')
reTitle = re.compile(r'<a.*?class="s xst".*?>([\s\S]+?)<\/a>')
reDate = re.compile(r'(\d{4}-\d+-\d+)')
rePost = re.compile(r'<td class="t_f" id="postmessage_\d+">([\s\S]*?)<\/td>')

In [58]:
def fetch(url):
    headers = {"Referer": "http://lbsbbs.amap.com/login.php?callback={}".format(url)}
    res = req.get(url, headers = headers)
    return res

In [64]:
def trip_tag(s):
    deTag = re.sub(r'<.*?>', '', s).strip()
    deBrack = re.sub(r'\[|\]', '', deTag)
    return deBrack

In [65]:
ENT_PAT = 'http://lbsbbs.amap.com/forum.php?mod=forumdisplay&fid=57&page={}'
DET_PAT = 'http://lbsbbs.amap.com/forum.php?mod=viewthread&tid={}'

In [70]:
for page in range(1, 381):
    page_url = ENT_PAT.format(page)
    
    res = fetch(page_url)
    text = res.text
    
    posts = []
    
    for m in reItem.finditer(text):
        post = {}
        
        tid = m.group("tid")
        item = m.group("item")
        titleTh = reTitleTh.findall(item)[0]
        ems = reEm.findall(titleTh)
        title = reTitle.findall(titleTh)[0]
        
        post['tid'] = tid
        post['url'] = DET_PAT.format(tid)
        post['title'] = title
        post['status'] = trip_tag(ems[0])
        if len(ems) > 1:
            post['tag'] = trip_tag(ems[1])
        
        post['date'] = reDate.findall(item)[0]
        posts.append(post)
    DB.insert_many(posts)

In [78]:
post = DB.find_one({"p": False})
while post:
    url = post['url']
    print(url)
    if url:
        res = fetch(url)
    
        content = rePost.findall(res.text)[0].strip()
    else:
        content = ""
    DB.update_one({"tid": post["tid"]}, {"$set": {"p": True, "content": content}})
    post = DB.find_one({"p": False})

http://lbsbbs.amap.com/forum.php?mod=viewthread&tid=19993


IndexError: list index out of range

In [79]:
import pandas as pd

In [85]:
df = pd.DataFrame(list(DB.find({"p":True}, {"_id": 0, "url": 0, "p": 0})))

In [86]:
df.head()

,content,date,status,tag,tid,title
0,高德导航车机版在不断升级，各种功能层出不穷，但各种问题越来越多，最后可能过半的车都不能用了。...,2017-12-31,求助中,导航,28850,个人给高德导航的建议
1,新版装上后跑了一天，遇到叮的一声音乐依旧变小不恢复，这问题咋老不解决，实在无语。原来用的别的...,2017-12-30,求助中,NaN,28844,叮的一声依旧变无声
2,升级到2.8.2.1363后发现几点缺陷：<br />\r\n1.不能后台（边导航边）下载离...,2018-1-1,求助中,导航,28879,2.8.2.1363几点缺陷
3,282众测版，导航设置里已经开启了自动缩放比例尺开关，在导航时却一直在50米比例尺级，不会自...,2017-12-31,求助中,导航,28866,282众测版导航比例尺问题
4,大屏分屏软件Auto_V1.2.2.2254_C04010001120&nbsp; &nbs...,2017-6-4,已解决,适配,25725,大屏分屏软件Auto_V1.2.2.2254_C04010001120


In [87]:
df.to_csv("amap.csv")

In [96]:
def clean_content(s):
    s = re.sub(r"<.*?>", "", s)
    s = re.sub(r"\s", " ", s)
    s = re.sub(r",", "，", s)
    return s

In [100]:
df['clean_content'] = df['content'].apply(clean_content)

In [103]:
df[['tid', 'status', 'tag', 'date', 'title', 'clean_content']].to_csv("amap.csv")